In [2]:
import pandas as pd
from scipy import stats
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import operator

In [4]:
food = pd.read_csv("datamarket_productos_de_supermercados.csv", encoding="utf-8")

In [5]:
food.head()

,url,supermarket,category,name,description,price,reference_price,reference_unit,insert_date,product_id
0,https://www.dia.es/compra-online/productos/per...,dia-es,perfumeria_e_higiene_colonias_femeninas,AQC FRAGANCES body mist corporal strawberry sp...,NaN,3.99,19.95,l,2021-03-06 00:00:00,f832dd6f8b3a2d85663f2a60502efe37
1,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_desnatada,ASTURIANA leche desnatada botella 1.5 lt,NaN,1.24,0.83,l,2021-03-06 00:00:00,c18fe61f99e7f4d7447e3d1d927f5ef2
2,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_semidesnatada,ASTURIANA leche semidesnatada envase 1 lt,NaN,0.79,0.79,l,2021-03-06 00:00:00,8ec2532134684e0c6590b89233b2f6c4
3,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_semidesnatada,ASTURIANA leche semidesnatada botella 2.2 lt,NaN,1.75,0.80,l,2021-03-06 00:00:00,6c738b9d8f185c3c1eda643516675a1c
4,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_semidesnatada,DIA DELICIOUS leche semidesnatada fresca botel...,NaN,1.18,0.79,l,2021-03-06 00:00:00,bd9f2aaa175a18cd7270ff244cbdbcbe


Hacemos una limpieza de todas las categorías de productos de supermercado que no son alimentos

In [6]:
len(food['category'].unique())

1503

In [7]:
blacklist = ['cocina','basura','detergente','electrodomestico','corporal','higienico','limpieza', 'jardineria', 
             'casa', 'hogar', 'cuidado', 'cabello', 'bebe',
             'perfumeria', 'mascota', 'parafarmacia', 'farmacia', 'maquillaje','drogueria','soy_solidario']

In [8]:
food = food[~food.category.str.contains('|'.join(blacklist))]

In [9]:
len(food['category'].unique())

953

In [10]:
list(food.category.unique())

['lacteos_y_huevos_leche_desnatada',
 'lacteos_y_huevos_leche_semidesnatada',
 'lacteos_y_huevos_leche_calcio',
 'lacteos_y_huevos_leche_sin_lactosa',
 'lacteos_y_huevos_leche_leches_especiales',
 'bebidas_refrescos_limon',
 'bebidas_aguas_agua_sin_gas',
 'bebidas_refrescos_tonica_y_ginger_ale',
 'bebidas_refrescos_bebidas_de_frutas',
 'bebidas_zumos_y_nectares_zumo',
 'bebidas_cervezas_estandar',
 'bebidas_refrescos_refrigerados',
 'bebidas_gaseosas_y_sodas_gaseosas_y_sodas',
 'bebidas_isotonicas_y_energeticas_isotonicas',
 'bebidas_zumos_y_nectares_nectar',
 'bebidas_zumos_y_nectares_refrigerados',
 'bebidas_zumos_y_nectares_mosto',
 'bebidas_alcoholes_y_licores_brandy_y_cognac',
 'bebidas_alcoholes_y_licores_cremas',
 'bebidas_alcoholes_y_licores_ginebra',
 'bebidas_alcoholes_y_licores_whisky_y_bourbon',
 'bebidas_alcoholes_y_licores_licor',
 'bebidas_alcoholes_y_licores_pacharan_y_ponche',
 'bebidas_alcoholes_y_licores_ron',
 'bebidas_vinos_blancos_rioja',
 'bebidas_vinos_blancos_v

In [11]:
food.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4339539 entries, 1 to 7295901
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   url              object 
 1   supermarket      object 
 2   category         object 
 3   name             object 
 4   description      object 
 5   price            float64
 6   reference_price  float64
 7   reference_unit   object 
 8   insert_date      object 
 9   product_id       object 
dtypes: float64(2), object(8)
memory usage: 364.2+ MB


In [12]:
food = food.drop(columns = ['product_id'])

In [13]:
food = food.rename(columns = {'insert_date' : 'date'})
food['date'] = food['date'].astype('datetime64[ns]')


In [14]:
food.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4339539 entries, 1 to 7295901
Data columns (total 9 columns):
 #   Column           Dtype         
---  ------           -----         
 0   url              object        
 1   supermarket      object        
 2   category         object        
 3   name             object        
 4   description      object        
 5   price            float64       
 6   reference_price  float64       
 7   reference_unit   object        
 8   date             datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 331.1+ MB


Para iniciar el procesamiento obtenemos los estimados de locación usando la funcion describe transpuesta usando solo los campos de precios

In [15]:
est_loc = food.describe().T

est_loc

,count,mean,std,min,25%,50%,75%,max
price,4339531.0,4.878172,63.944793,0.080,1.19,1.89,3.00,7915.05
reference_price,4339533.0,10.911581,126.916493,0.008,2.37,5.32,9.99,16633.33


In [16]:
variables_numericas = ['price','reference_price']
columnas = ['promedio', 'mediana', 'media_truncada', 'desviacion_estandar', 'rango', '25%', '50%', '75%', 'rango_intercuartil']

estimados_loc = pd.DataFrame(index = variables_numericas)

datos = []

for columna in columnas:
    if columna == 'promedio':
        for i in range(len(variables_numericas)):
            datos.append(food[variables_numericas[i]].mean())
        estimados_loc.insert(0, column = columna, value = datos)
    elif columna == 'mediana':
        for i in range(len(variables_numericas)):
            datos.append(food[variables_numericas[i]].median())
        estimados_loc.insert(1, column = columna, value = datos)
    elif columna == 'media_truncada':
        for i in range(len(variables_numericas)):
            datos.append(stats.trim_mean(food[variables_numericas[i]], 0.1))
        estimados_loc.insert(2, column = columna, value = datos)
    elif columna == 'desviacion_estandar':
        for i in range(len(variables_numericas)):
            datos.append(food[variables_numericas[i]].std())
        estimados_loc.insert(3, column = columna, value = datos)
    elif columna == 'rango':
        for i in range(len(variables_numericas)):
            datos.append(food[variables_numericas[i]].max() - food[variables_numericas[i]].min())
        estimados_loc.insert(4, column = columna, value = datos)
    elif columna == '25%':
        for i in range(len(variables_numericas)):
            datos.append(food[variables_numericas[i]].quantile(0.25))
        estimados_loc.insert(5, column = columna, value = datos)
    elif columna == '50%':
        for i in range(len(variables_numericas)):
            datos.append(food[variables_numericas[i]].quantile(0.5))
        estimados_loc.insert(6, column = columna, value = datos)
    elif columna == '75%':
        for i in range(len(variables_numericas)):
            datos.append(food[variables_numericas[i]].quantile(0.75))
        estimados_loc.insert(7, column = columna, value = datos)
    elif columna == 'rango_intercuartil':
        for i in range(len(variables_numericas)):
            datos.append(food[variables_numericas[i]].quantile(0.75) - food[variables_numericas[i]].quantile(0.25))
        estimados_loc.insert(8, column = columna, value = datos)
    datos = []

estimados_loc

,promedio,mediana,media_truncada,desviacion_estandar,rango,25%,50%,75%,rango_intercuartil
price,4.878172,1.89,2.141271,63.944793,7914.970,1.19,1.89,3.00,1.81
reference_price,10.911581,5.32,6.247450,126.916493,16633.322,2.37,5.32,9.99,7.62


Tanto del precio total de producto como del precio de referencia tenemos mucha diferencia del tercer cuartil al valor máximo

In [17]:
food_1000 = food[food['price'] > 1000]
food_1000.sort_values(by=['price'], ascending=False)

,url,supermarket,category,name,description,price,reference_price,reference_unit,date
1860902,https://tienda.mercadona.es/product/24115/cara...,mercadona-es,congelados_marisco,Carabinero congelado,Granel,7915.05,79.95,kg,2021-06-29
1502646,https://tienda.mercadona.es/product/24115/cara...,mercadona-es,congelados_marisco,Carabinero congelado,Granel,7915.05,79.95,kg,2021-06-05
1493322,https://tienda.mercadona.es/product/24115/cara...,mercadona-es,congelados_marisco,Carabinero congelado,Granel,7915.05,79.95,kg,2021-06-04
1464639,https://tienda.mercadona.es/product/24115/cara...,mercadona-es,congelados_marisco,Carabinero congelado,Granel,7915.05,79.95,kg,2021-06-03
2260345,https://tienda.mercadona.es/product/24115/cara...,mercadona-es,congelados_marisco,Carabinero congelado,Granel,7915.05,79.95,kg,2021-07-26
...,...,...,...,...,...,...,...,...,...
1913236,https://tienda.mercadona.es/product/24147/gamb...,mercadona-es,congelados_marisco,Gambón congelado,Granel,1039.50,10.50,kg,2021-07-03
4257073,https://tienda.mercadona.es/product/24147/gamb...,mercadona-es,congelados_marisco,Gambón congelado,Granel,1039.50,10.50,kg,2021-12-12
2732918,https://tienda.mercadona.es/product/24147/gamb...,mercadona-es,congelados_marisco,Gambón congelado,Granel,1039.50,10.50,kg,2021-08-28
3439011,https://tienda.mercadona.es/product/24147/gamb...,mercadona-es,congelados_marisco,Gambón congelado,Granel,1039.50,10.50,kg,2021-10-18


In [18]:
food['date']

1         2021-03-06
2         2021-03-06
3         2021-03-06
4         2021-03-06
5         2021-03-06
             ...    
7295897   2022-07-10
7295898   2022-07-10
7295899   2022-07-10
7295900   2022-07-10
7295901   2022-07-10
Name: date, Length: 4339539, dtype: datetime64[ns]

Los productos más caros son a granel ya que toma el precio de 99 kilos del producto, procedemos a solo usar el precio de un kilo para regularizar

In [16]:
#muestra de un día de productos
food_sample = food[food['date'] == "2022-06-20"]
food_sample

,url,supermarket,category,name,description,price,reference_price,reference_unit,date
7005466,https://www.carrefour.es/supermercado/pechuga-...,carrefour-es,productos_frescos_charcuteria_fiambre_de_pavo_...,Pechuga de pollo en finas lonchas Carrefour 200 g,NaN,1.75,8.75,kg,2022-06-20
7005915,https://www.carrefour.es/supermercado/sidra-me...,carrefour-es,bebidas_sidra_natural,Sidra Menendez natural pack de 6 botellas de 7...,NaN,11.85,2.82,l,2022-06-20
7005916,https://www.carrefour.es/supermercado/sidra-ec...,carrefour-es,bebidas_sidra_natural,Sidra Ecológica Menendez natural 70 cl.,NaN,2.55,3.64,l,2022-06-20
7005917,https://www.carrefour.es/supermercado/sidra-de...,carrefour-es,bebidas_sidra_natural,De Nuestra Tierra Natural,NaN,2.40,3.43,l,2022-06-20
7005918,https://www.carrefour.es/supermercado/sidra-me...,carrefour-es,bebidas_sidra_natural,Menendez Natural,NaN,1.95,2.79,l,2022-06-20
...,...,...,...,...,...,...,...,...,...
7019858,https://www.carrefour.es/supermercado/nata-liq...,carrefour-es,la_despensa_lacteos_nata,Nata liquida para montar Carrefour pack de 3 u...,NaN,2.09,3.48,l,2022-06-20
7019859,https://www.carrefour.es/supermercado/nata-par...,carrefour-es,la_despensa_lacteos_nata,Nata para montar líquida Carrefour 500 ml.,NaN,1.24,2.48,l,2022-06-20
7019860,https://www.carrefour.es/supermercado/nata-car...,carrefour-es,la_despensa_lacteos_nata,Nata Carrefour 200 ml.,NaN,0.85,4.25,l,2022-06-20
7019861,https://www.carrefour.es/supermercado/leche-co...,carrefour-es,la_despensa_lacteos_leche_condensada_evaporada...,Leche condensada Carrefour 450 g.,NaN,1.55,3.44,kg,2022-06-20


In [17]:
#en la muestra tenemos el mismo problema
est_loc = food_sample.describe().T

est_loc

,count,mean,std,min,25%,50%,75%,max
price,8528.0,5.344061,57.083355,0.110,1.29,2.00,3.250,2623.5
reference_price,8528.0,11.029559,111.572185,0.017,2.52,5.75,10.685,6375.0


In [18]:
food_sample[food_sample["description"] == "Granel" ]
#el problema se repite con productos donde también se toman solo el minimo de gramos 
#así que podemos estandarizar todo lo de granel al precio de referencia

,url,supermarket,category,name,description,price,reference_price,reference_unit,date
7007988,https://tienda.mercadona.es/product/24487/boca...,mercadona-es,congelados_marisco,Bocas de cangrejo pequeñas congeladas,Granel,1960.20,19.80,kg,2022-06-20
7010617,https://tienda.mercadona.es/product/24167/cuer...,mercadona-es,congelados_marisco,Cuerpos chicos de cangrejo Dimarosa congelados,Granel,1633.50,16.50,kg,2022-06-20
7010625,https://tienda.mercadona.es/product/24135/lang...,mercadona-es,congelados_marisco,Langostino crudo ultracongelado,Granel,940.50,9.50,kg,2022-06-20
7010626,https://tienda.mercadona.es/product/24105/alis...,mercadona-es,congelados_marisco,Alistado mediano congelado,Granel,2623.50,26.50,kg,2022-06-20
7010627,https://tienda.mercadona.es/product/24108.1/al...,mercadona-es,congelados_marisco,Alistado pequeño congelado,Granel,1514.70,15.30,kg,2022-06-20
7010628,https://tienda.mercadona.es/product/24147/gamb...,mercadona-es,congelados_marisco,Gambón congelado,Granel,1188.00,12.00,kg,2022-06-20
7010629,https://tienda.mercadona.es/product/24122/gamb...,mercadona-es,congelados_marisco,Gambón grande congelado,Granel,955.35,9.65,kg,2022-06-20
7011702,https://tienda.mercadona.es/product/3639/cerez...,mercadona-es,fruta_y_verdura_fruta,Cereza,Granel,573.21,5.79,kg,2022-06-20
7013755,https://tienda.mercadona.es/product/24247/tira...,mercadona-es,marisco_y_pescado_pescado_fresco,Tiras de potón,Granel,490.05,4.95,kg,2022-06-20
7014362,https://tienda.mercadona.es/product/24100.1/ga...,mercadona-es,congelados_marisco,Gamba arrocera congelada,Granel,1282.05,12.95,kg,2022-06-20


In [19]:
food_sample.loc[food_sample.description=="Granel",'price']= food_sample.reference_price

NameError: name 'food_sample' is not defined

In [20]:
#intentamos limpiar todo el dataser
food.loc[food.description=="Granel","price"]= food.reference_price

In [21]:
food[food["description"] == "Granel" ]

,url,supermarket,category,name,description,price,reference_price,reference_unit,date
3966,https://tienda.mercadona.es/product/24247/tira...,mercadona-es,marisco_y_pescado_pescado_fresco,Tiras de potón descongeladas,Granel,5.95,5.95,kg,2021-03-06
5076,https://tienda.mercadona.es/product/24167/cuer...,mercadona-es,congelados_marisco,Cuerpos chicos de cangrejo Dimarosa congelados,Granel,14.50,14.50,kg,2021-03-06
5078,https://tienda.mercadona.es/product/24168/boca...,mercadona-es,congelados_marisco,Bocas chicas de cangrejos congeladas,Granel,15.50,15.50,kg,2021-03-06
15245,https://tienda.mercadona.es/product/24147/gamb...,mercadona-es,congelados_marisco,Gambón congelado,Granel,9.50,9.50,kg,2021-03-06
15248,https://tienda.mercadona.es/product/24108.1/al...,mercadona-es,congelados_marisco,Alistado pequeño congelado,Granel,13.55,13.55,kg,2021-03-06
...,...,...,...,...,...,...,...,...,...
7292389,https://tienda.mercadona.es/product/24105/alis...,mercadona-es,congelados_marisco,Alistado mediano congelado,Granel,26.50,26.50,kg,2022-07-10
7292390,https://tienda.mercadona.es/product/24135/lang...,mercadona-es,congelados_marisco,Langostino crudo ultracongelado,Granel,9.50,9.50,kg,2022-07-10
7292426,https://tienda.mercadona.es/product/24100.1/ga...,mercadona-es,congelados_marisco,Gamba arrocera congelada,Granel,12.95,12.95,kg,2022-07-10
7292427,https://tienda.mercadona.es/product/24169/gamb...,mercadona-es,congelados_marisco,Gamba blanca congelada,Granel,19.45,19.45,kg,2022-07-10


In [22]:
food_sample[food_sample["description"] == "Granel" ]

NameError: name 'food_sample' is not defined

In [23]:
food.describe().T

,count,mean,std,min,25%,50%,75%,max
price,4339531.0,3.083783,8.947227,0.080,1.19,1.89,3.00,479.75
reference_price,4339533.0,10.911581,126.916493,0.008,2.37,5.32,9.99,16633.33


In [24]:
food.sort_values(by=['price'], ascending=False)

,url,supermarket,category,name,description,price,reference_price,reference_unit,date
6724813,https://tienda.mercadona.es/product/58323/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón de bellota ibérico 100% Campo Extremadura,Pieza,479.75,50.50,kg,2022-05-31
7240468,https://tienda.mercadona.es/product/58323/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón de bellota ibérico 100% Campo Extremadura,Pieza,479.75,50.50,kg,2022-07-07
6860971,https://tienda.mercadona.es/product/58323/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón de bellota ibérico 100% Campo Extremadura,Pieza,479.75,50.50,kg,2022-06-09
6744839,https://tienda.mercadona.es/product/58323/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón de bellota ibérico 100% Campo Extremadura,Pieza,479.75,50.50,kg,2022-06-01
6804224,https://tienda.mercadona.es/product/58323/jamo...,mercadona-es,charcuteria_y_quesos_jamon_serrano,Jamón de bellota ibérico 100% Campo Extremadura,Pieza,479.75,50.50,kg,2022-06-06
...,...,...,...,...,...,...,...,...,...
7105836,https://www.dia.es/compra-online/despensa/desa...,dia-es,despensa_desayunos_y_dulces_bolleria,BISCUITS GALICIA mini magdalenas con mantequil...,NaN,NaN,NaN,NaN,2022-06-27
7111129,https://www.dia.es/compra-online/despensa/desa...,dia-es,despensa_desayunos_y_dulces_bolleria,BISCUITS GALICIA mini magdalenas con mantequil...,NaN,NaN,NaN,NaN,2022-06-28
7112112,https://www.dia.es/compra-online/despensa/past...,dia-es,despensa_pastas_harinas_y_masas_pastas,DIA SELECCIÓN MUNDIAL tortiglioni paquete 500 gr,NaN,NaN,NaN,NaN,2022-06-28
7120481,https://www.dia.es/compra-online/despensa/lact...,dia-es,despensa_lacteos_y_huevos_mantequilla_y_margarina,DIA margarina ligera barqueta 500 gr,NaN,NaN,NaN,NaN,2022-06-28


Ahora revisamos los valores de referencia elevados para ver si son congruentes

In [25]:
food[food['reference_price'] > 2000].sort_values(by=['price'], ascending=False)

,url,supermarket,category,name,description,price,reference_price,reference_unit,date
4686359,https://www.dia.es/compra-online/congelados/pe...,dia-es,congelados_pescado_y_marisco,DIMAR lomos de bacalao sin piel envase 300 gr,NaN,4.99,16633.33,NaN,2022-01-11
4675272,https://www.dia.es/compra-online/congelados/pe...,dia-es,congelados_pescado_y_marisco,DIMAR lomos de bacalao sin piel envase 300 gr,NaN,4.99,16633.33,NaN,2022-01-10
4666139,https://www.dia.es/compra-online/congelados/pe...,dia-es,congelados_pescado_y_marisco,DIMAR lomos de bacalao sin piel envase 300 gr,NaN,4.99,16633.33,NaN,2022-01-09
310978,https://www.dia.es/compra-online/productos/ali...,dia-es,alimentacion_sal_especias_y_sazonadores_especias,POTE azafrán en hebras caja 4 sobres,NaN,3.29,8225.00,kg,2021-03-21
280732,https://www.dia.es/compra-online/productos/ali...,dia-es,alimentacion_sal_especias_y_sazonadores_especias,POTE azafrán en hebras caja 4 sobres,NaN,3.29,8225.00,kg,2021-03-20
...,...,...,...,...,...,...,...,...,...
7217606,https://www.dia.es/compra-online/bebidas/cerve...,dia-es,bebidas_cervezas_nacionales,MAHOU 5 ESTRELLAS cerveza lata 33 cl,NaN,0.75,2272.73,l,2022-07-05
7277911,https://www.dia.es/compra-online/bebidas/cerve...,dia-es,bebidas_cervezas_nacionales,MAHOU 5 ESTRELLAS cerveza lata 33 cl,NaN,0.75,2272.73,l,2022-07-09
7282767,https://www.dia.es/compra-online/bebidas/cerve...,dia-es,bebidas_cervezas_nacionales,MAHOU 5 ESTRELLAS cerveza lata 33 cl,NaN,0.75,2272.73,l,2022-07-10
7241200,https://www.dia.es/compra-online/bebidas/cerve...,dia-es,bebidas_cervezas_nacionales,MAHOU 5 ESTRELLAS cerveza lata 33 cl,NaN,0.75,2272.73,l,2022-07-07


In [26]:
# tabla de frecuencia 
pd.value_counts(food['category'])


congelados_helados                                               71648
bodega_vino_tinto                                                63446
huevos_leche_y_mantequilla_leche_y_bebidas_vegetales             59541
azucar_caramelos_y_chocolate_chocolate                           52391
bodega_cerveza                                                   50798
                                                                 ...  
el_mercado_carniceria_casqueria                                      9
productos_frescos_sushi_del_dia_quesos                               6
el_mercado_frutas_uvas                                               5
bebidas_finos_y_dulces_moscatel                                      1
productos_frescos_sushi_del_dia_charcuteria_y_quesos_al_corte        1
Name: category, Length: 953, dtype: int64

In [27]:
# tabla de frecuencia relativa 
100 * food['category'].value_counts() / len(food['category'])

congelados_helados                                               1.651051
bodega_vino_tinto                                                1.462045
huevos_leche_y_mantequilla_leche_y_bebidas_vegetales             1.372058
azucar_caramelos_y_chocolate_chocolate                           1.207294
bodega_cerveza                                                   1.170585
                                                                   ...   
el_mercado_carniceria_casqueria                                  0.000207
productos_frescos_sushi_del_dia_quesos                           0.000138
el_mercado_frutas_uvas                                           0.000115
bebidas_finos_y_dulces_moscatel                                  0.000023
productos_frescos_sushi_del_dia_charcuteria_y_quesos_al_corte    0.000023
Name: category, Length: 953, dtype: float64

In [26]:
food_sample.to_csv("supermercados_muestra_limpia_20-06-2022.csv")

In [33]:
food.to_csv("supermercados_completo_limpio_10-07-2022.csv", encoding="utf-8")

,url,supermarket,category,name,description,price,reference_price,reference_unit,date
6,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_semidesnatada,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.15,1.15,l,2021-03-06
32127,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_semidesnatada,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.15,1.15,l,2021-03-07
55043,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_semidesnatada,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.15,1.15,l,2021-03-08
72636,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_semidesnatada,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.15,1.15,l,2021-03-09
81260,https://www.dia.es/compra-online/productos/lac...,dia-es,lacteos_y_huevos_leche_semidesnatada,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.15,1.15,l,2021-03-10
...,...,...,...,...,...,...,...,...,...
4774087,https://www.dia.es/compra-online/despensa/lact...,dia-es,despensa_lacteos_y_huevos_leche,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.10,1.10,l,2022-01-17
4788172,https://www.dia.es/compra-online/despensa/lact...,dia-es,despensa_lacteos_y_huevos_leche,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.10,1.10,l,2022-01-18
4805776,https://www.dia.es/compra-online/despensa/lact...,dia-es,despensa_lacteos_y_huevos_leche,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.10,1.10,l,2022-01-19
4824851,https://www.dia.es/compra-online/despensa/lact...,dia-es,despensa_lacteos_y_huevos_leche,CELTA leche semidesnatada Bio ecológica envase...,NaN,1.10,1.10,l,2022-01-20
